In [3]:
import pickle
import os
import csv
import numpy as np
os.chdir('/Users/david/Desktop/FinetuneEmbed')
from mod.mod_text import *

data_dir = "./data/long_vs_shortTF"
save_csv_dir = "./res/2024_1125/LongShortTF/long_vs_shortTF_finetune_auc.csv"
output_path = "./res/2024_1125/LongShortTF/LongShortTF_model_"

model_name = "sentence-transformers/all-MiniLM-L6-v2"
mean_val_auc_ls = []
test_auc_ls = []

random_states = list(range(41, 51)) # set up the random seeds
random_states = [1, 2]
random_state = 1

In [4]:
output_dir = output_path  + str(random_state)


train_dir = data_dir + "/TrainEvalTestData/train_data_" + str(random_state) + ".pkl"
eva_dir = data_dir + "/TrainEvalTestData/eval_data_" + str(random_state) + ".pkl"
test_dir = data_dir + "/TrainEvalTestData/test_data_" + str(random_state) + ".pkl"

# prepare the input data
with open(train_dir, "rb") as f:
    train_data = pickle.load(f)
with open(eva_dir, "rb") as f:
    eval_data = pickle.load(f)
with open(test_dir, "rb") as f:
    test_data = pickle.load(f)

val_auc_scores = []
output_dirs = []  # Track output directories for each fold

train_texts, train_labels = train_data['desc'], train_data['labels']
eval_texts, eval_labels = eval_data['desc'], eval_data['labels']
test_texts, test_labels = test_data['desc'], test_data['labels']

tokenizer = AutoTokenizer.from_pretrained(model_name)


In [8]:
mod_dir, val_auc = one_fold_training(train_texts, train_labels, eval_texts, eval_labels, 
                                     tokenizer, output_dir, 0, model_name)

/Users/david/anaconda3/envs/FinetuneBERT/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  5%|▌         | 13/260 [00:02<00:36,  6.73it/s]

{'eval_loss': 0.5997536778450012, 'eval_AUC': 0.3504273504273504, 'eval_runtime': 0.1597, 'eval_samples_per_second': 219.198, 'eval_steps_per_second': 31.314, 'epoch': 1.0}


 10%|█         | 26/260 [00:04<00:35,  6.67it/s]

{'eval_loss': 0.5722935199737549, 'eval_AUC': 0.36752136752136755, 'eval_runtime': 0.1566, 'eval_samples_per_second': 223.562, 'eval_steps_per_second': 31.937, 'epoch': 2.0}


 15%|█▌        | 39/260 [00:07<00:33,  6.52it/s]

...

{'eval_loss': 1.6445518732070923, 'eval_AUC': 0.423076923076923, 'eval_runtime': 0.1718, 'eval_samples_per_second': 203.743, 'eval_steps_per_second': 29.106, 'epoch': 13.0}


 65%|██████▌   | 169/260 [00:34<00:18,  4.95it/s]


{'train_runtime': 34.1591, 'train_samples_per_second': 60.891, 'train_steps_per_second': 7.611, 'train_loss': 0.3730248445589867, 'epoch': 13.0}


100%|██████████| 5/5 [00:00<00:00, 52.24it/s]

Fold 1 Validation AUC: 0.47435897435897434


In [9]:

val_auc_scores.append(val_auc)
output_dirs.append(mod_dir)

In [16]:
# locate the best model
best_fold_idx = np.argmax(val_auc_scores)
best_model_dir = output_dir + '/fold_' + str(best_fold_idx + 1)
best_model = AutoModelForSequenceClassification.from_pretrained(best_model_dir)

# create the training data for final training
train_texts_all = train_texts + eval_texts
train_labels_all = train_labels + eval_labels

In [17]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
full_train_dataset = TextDataset(train_texts_all, train_labels_all, tokenizer)

# Fine-tune the best model on the full training data
output_dir = output_dir + "/final_model"
final_trainer = finetune_best_mod(full_train_dataset, best_model, output_dir)

# Evaluate the best model on the test set
test_dataset = TextDataset(test_texts, test_labels, tokenizer)
test_auc = pred_test(final_trainer, test_dataset)

/Users/david/anaconda3/envs/FinetuneBERT/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
100%|██████████| 90/90 [00:14<00:00,  6.31it/s]


{'train_runtime': 14.2582, 'train_samples_per_second': 48.744, 'train_steps_per_second': 6.312, 'train_loss': 0.32301436530219185, 'epoch': 5.0}


100%|██████████| 5/5 [00:00<00:00, 48.14it/s]

Test AUC: 0.7264957264957266
